# Basic Example Notebook
Hello and welcome to `PauliPropagation.jl` - a Julia package for Pauli propagation simulation of quantum circuits and quantum systems.

This package can be used to estimate quantities like
$ \text{Tr}[\rho \ \mathcal{E}(O)] $
where $\rho$ is an initial state, $\mathcal{E}$ is a quantum channel that is not necessarily unitary, and $O$ is an observable preferrably sparse in Pauli basis.

In the following we are going to introduce the basic concepts and datatypes of the package.

In [1]:
using PauliPropagation

Define the number of qubits in the simulation.

In [2]:
nq = 64

64

Now define the observable $O = Z_{32} = I_1 \otimes ... \otimes I_{31} \otimes Z_{32} \otimes Z_{33} \otimes I_{33}... \otimes I_{64}$.

The `PauliString` type is our high-level way of handling Pauli strings. This above constructor defines a `PauliString` that has the only non-Identity Pauli at position 32.

This library uses encodes Paulis into the bits of Integers for performance reasons, and also actions of gates onto Paulis are defined as bit operations. You can definitely get by just sticking to the high level interface. Check out some other example notebooks for more involved code.

In [3]:
pstr = PauliString(nq, :Z, 32)

PauliString(nqubits: 64, 1.0 * IIIIIIIIIIIIIIIIIIII...)

As another example, this what it prints for a 4-qubit example with a different coefficient:

In [4]:
PauliString(4, [:X, :Y, :Z], [2, 3, 4], 2.0)

PauliString(nqubits: 4, 2.0 * IXYZ)

Under the hood everything works with the `PauliSum` data type. It can also be passed to `propagate()`, and you can create it like this:

In [5]:
# input_psum = PauliSum(nq)
# add!(input_psum, :Z, 32)
# add!(input_psum, [:X, :X], [1, 64])
# add!(input_psum, [:X, :Y, :Z], [1, 4, 7], 1.3)

Now we specify the circuit that we want to run. We provide some simple constructors for common circuit ansätze. Here we use a generic circuit ansatz called the `hardwareefficientcircuit`. It consists of repeated RX- RZ - RX Pauli rotation gates intertwined with RYY entangling gates.

For everything to work out, we need to specify the number of circuit layers and the entangling topology. By default we use the so-called 1D `bricklayertopology`. 

In [5]:
nl = 6
topology = bricklayertopology(nq; periodic=false)
circuit = hardwareefficientcircuit(nq, nl; topology=topology)
nparams = countparameters(circuit)

1530

But you can use other predefined topologies in the library or customize a topology via an array or tuples like `[(1, 2), (1, 3), ...]`.

In [7]:
# rectopology = rectangletopology(8, 8; periodic=false) 

# hardcoded_custom_topology = [(1, 2), (1, 15), (2, 3), (3, 4), (4, 5), (5, 6), (5, 16), (6, 7)]

# function centralspintopology(nqubits)
#     return [(1, ii) for ii in 2:nqubits]
# end

# centralspin_topology = centralspintopology(nq)

There are also several other inbuilt circuits in the library and you can define your own custom circuits. Note that both circuits, and the angles they take, are defined in the Schrodinger picture. For example:

In [8]:
# circuit = tfitrottercircuit(nq, nl; topology=topology, start_with_ZZ=true);
# circuit = tiltedtfitrottercircuit(nq, nl; topology=topology)
# circuit = heisenbergtrottercircuit(nq, nl; topology=topology)

In [9]:
# function myheisenbergtrottercircuit(nqubits::Integer, nlayers::Integer; topology=nothing)
#     circuit::Vector{Gate} = []

#     if isnothing(topology)
#         topology = bricklayertopology(nqubits)
#     end

#     for _ in 1:nlayers
#         rxxlayer!(circuit, topology)
#         ryylayer!(circuit, topology)
#         rzzlayer!(circuit, topology)
#     end

#     return circuit
# end

# circuit = myheisenbergtrottercircuit(nq, nl; topology=topology);
# nparams = countparameters(circuit)

# Jxx = 0.5
# Jyy = 0.25
# Jzz = 0.25
# dt = 0.2

# thetaxx_indices = getparameterindices(circuit, PauliRotation, [:X, :X])
# thetayy_indices = getparameterindices(circuit, PauliRotation, [:Y, :Y])
# thetazz_indices = getparameterindices(circuit, PauliRotation, [:Z, :Z])

# thetas = zeros(nparams)
# thetas[thetaxx_indices] .= 2 * Jxx * dt
# thetas[thetayy_indices] .= 2 * Jyy * dt
# thetas[thetazz_indices] .= 2 * Jzz * dt;

Let us draw some random parameters from a narrow-ish Gaussian around zero.
These parameters are 1-to-1 assigned to the parametrized gates in the circuit, for the beginning to the end. Internally we will reverse both and apply the gates with their corresponding parameter in the Heisenberg picture.

In [6]:
using Random
Random.seed!(42)
thetas = ones(nparams) * 0.1;

Now we get to the interesting part. The propagation of the Observable `pstr` through `circuit`. In other words, we apply the circuit onto the Pauli string.

First, let us do it exactly.

In [ ]:
psum = propagate(circuit, pstr, thetas, min_abs_coeff = 0)

Just to run it again after Julia's jit (just-in-time)-compilation, which compiles the code and makes subsequent calls faster:

In [12]:
@time propagate(circuit, pstr, thetas);

MethodError: MethodError: no method matching set!(::PauliSum{UInt128, Float64}, ::UInt128, ::ComplexF64)

Closest candidates are:
  set!(!Matched::Dict{TT, CT}, ::TT, ::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:639
  set!(::PauliSum{TT, CT}, ::TT, !Matched::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:633
  set!(!Matched::Dict{TT, NodePathProperties}, ::TT, !Matched::NodePathProperties) where TT
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\Surrogate\datatypes.jl:126


How is it possible that we can exactly run 64-qubit circuits? We leverage a concept referred to as the _entanglement lightcone_. But we don't do it manually, the Pauli propagation framework does it naturally. Still, we created over 160k Pauli strings in an object of type `PauliSum`. As the name suggests, this is a sum of Pauli strings. Be very mindful of the fact that the runtime will initially scale exponentially with the circuit depth!

The `psum` object may already be of interest of you. But here is a quick and easy way how to evaluate the expectation value if initial state is the zero-state, i.e. $\rho = |0\rangle\langle 0|$.

In [13]:
overlapwithzero(psum)

UndefVarError: UndefVarError: `psum` not defined

Remember, this was an exact calculation. Well actually, our library has a default truncation threshold of Pauli strings with coefficients smaller than 1e-10. Here is how you control the minimum absolute coefficient threshold `min_abs_coeff`. (Warning - if you are now using your own custom circuit / topology this may time out). 

In [14]:
@time psum_exact = propagate(circuit, pstr, thetas; min_abs_coeff=0)

MethodError: MethodError: no method matching set!(::PauliSum{UInt128, Float64}, ::UInt128, ::ComplexF64)

Closest candidates are:
  set!(!Matched::Dict{TT, CT}, ::TT, ::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:639
  set!(::PauliSum{TT, CT}, ::TT, !Matched::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:633
  set!(!Matched::Dict{TT, NodePathProperties}, ::TT, !Matched::NodePathProperties) where TT
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\Surrogate\datatypes.jl:126


And the expectation value is pretty much the same.

In [15]:
print("The error with our default truncation of `1e-10` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum)))

UndefVarError: UndefVarError: `psum_exact` not defined

How about higher truncation thresholds?

In [16]:
min_abs_coeff = 1e-3
@time psum_coeff = propagate(circuit, pstr, thetas; min_abs_coeff = min_abs_coeff)

MethodError: MethodError: no method matching set!(::PauliSum{UInt128, Float64}, ::UInt128, ::ComplexF64)

Closest candidates are:
  set!(!Matched::Dict{TT, CT}, ::TT, ::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:639
  set!(::PauliSum{TT, CT}, ::TT, !Matched::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:633
  set!(!Matched::Dict{TT, NodePathProperties}, ::TT, !Matched::NodePathProperties) where TT
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\Surrogate\datatypes.jl:126


In [17]:
print("The error with `min_abs_coeff = $min_abs_coeff` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum_coeff)))

UndefVarError: UndefVarError: `psum_exact` not defined

The expectation value is still surprisingly precise.

As a side-note, something that can be very useful and efficient, is to change a `PauliSum` *in-place*. For example:

In [18]:
# input_psum = PauliSum(nq)
# add!(input_psum, :Z, 32)
# add!(input_psum, [:X, :X], [1, 64])
# add!(input_psum, [:X, :Y, :Z], [1, 4, 7], 1.3)

# @time propagate!(circuit, input_psum, thetas; min_abs_coeff=1e-3)
# @time propagate!(circuit, input_psum, thetas; min_abs_coeff=1e-3) # and again
# input_psum

Another very general but powerful truncation is one based on _Pauli weight_. We can truncate Pauli strings that have many non-Identity Paulis, which has been proven to be a valid truncation for random circuits, but it also works well in practice. We can pass this as the keyword argument `max_weight`.

In [19]:
max_weight = 5
@time psum_weight = propagate(circuit, pstr, thetas; max_weight = max_weight)

MethodError: MethodError: no method matching set!(::PauliSum{UInt128, Float64}, ::UInt128, ::ComplexF64)

Closest candidates are:
  set!(!Matched::Dict{TT, CT}, ::TT, ::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:639
  set!(::PauliSum{TT, CT}, ::TT, !Matched::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:633
  set!(!Matched::Dict{TT, NodePathProperties}, ::TT, !Matched::NodePathProperties) where TT
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\Surrogate\datatypes.jl:126


In [20]:
print("The error with `max_weight = $max_weight` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum_weight)))

UndefVarError: UndefVarError: `psum_exact` not defined

One mindset to adopt when using this package (or any other computational physics package for that matter), is that exact computation will quickly be infeasible. Truncations introduce a necessary trade-off between computational cost and accuracy.

### Time vs Accuracy

Truncations are absolutely vital for Pauli propagation. Choose the truncations too tight and the results will be systematically wrong, choose the truncations too loose and the computation will be too hard. 

Here we show how varying the minimal coefficient threshold `min_abs_coeff` affects both the error and the runtime.

In [21]:
# using Pkg; Pkg.add("Plots")

using Plots
Plots.scalefontsizes(1.2)

In [22]:
# the min_abs_coeff to check
trunc_coeffs = 10.0 .^ (-1:-1:-10)

# One more exact computation. Record the expectation value and time.
res = @timed propagate(circuit, pstr, thetas; min_abs_coeff=0)
exact_expectation = overlapwithzero(res.value)
exact_time = res.time

# Sweep over the truncation values
expectations = zeros(length(trunc_coeffs))
times = zeros(length(trunc_coeffs))
for (ii, min_abs_coeff) in enumerate(trunc_coeffs)
    res = @timed propagate(circuit, pstr, thetas; min_abs_coeff=min_abs_coeff)
    expectations[ii] = overlapwithzero(res.value)
    times[ii] = res.time
end

# calculate the absolute error to the exact expectation value
abs_errors = abs.(expectations .- exact_expectation);

MethodError: MethodError: no method matching set!(::PauliSum{UInt128, Float64}, ::UInt128, ::ComplexF64)

Closest candidates are:
  set!(!Matched::Dict{TT, CT}, ::TT, ::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:639
  set!(::PauliSum{TT, CT}, ::TT, !Matched::CT) where {TT, CT}
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\PauliAlgebra\datatypes.jl:633
  set!(!Matched::Dict{TT, NodePathProperties}, ::TT, !Matched::NodePathProperties) where TT
   @ PauliPropagation C:\Users\andre\Desktop\EPFL\Master-Project\ImaginaryTimeEvolutionPauliPropagation\src\Surrogate\datatypes.jl:126


A plot of truncation threshold vs error (lower is better):

In [23]:
plot(trunc_coeffs, abs_errors, xscale=:log10, yscale=:log10, marker=:o, label="With coefficient truncation", xticks=sort(trunc_coeffs), xlabel="Truncation Threshold", ylabel="Error", yticks=10.0 .^ (-10:1:0), legend=:topleft)

UndefVarError: UndefVarError: `abs_errors` not defined

A plot of truncation threshold vs runtime in seconds (lower is better):

In [24]:
plot(trunc_coeffs, times, xscale=:log10, yscale=:log10, marker=:o, label="With coefficient truncation", xticks=sort(trunc_coeffs), ylabel="Runtime [s]", xlabel="Truncation Threshold", yticks=10.0 .^ (-4:1:0), legend=:bottomleft)
hline!([exact_time], color="grey", label="Exact computation", linestyle=:dash, lw=2)

UndefVarError: UndefVarError: `times` not defined

And finally, plot the error against the runtime. This tells you, if you need the result to `x` accuraccy, you will need `y` time.  

In [25]:
plot(abs_errors, times, xscale=:log10, marker=:o, yscale=:log10, label="With coefficient truncation", xticks=10.0 .^ (-10:2:0), xlabel="Error", ylabel="Runtime [s]", yticks=10.0 .^ (-4:1:0), legend=:bottomleft)
hline!([exact_time], color="grey", label="Exact computation", linestyle=:dash, lw=2)

UndefVarError: UndefVarError: `abs_errors` not defined

This highlights a very important concept: There is a time vs accuracy trade-off where one can get away with orders of magnitude less computing resources compared to exact results if one is happy with a certain error. The problem: You may never know what the exact result is, and thus you won't truly know how accurate the results are. You have to check for "convergence", which is a stabilization of the result as you lower the truncation. See for example this plot of the expectation value without the exact value for reference:

In [26]:
plot(trunc_coeffs, expectations, xscale=:log10, marker=:o, label="", xticks=sort(trunc_coeffs), xlabel="Truncation Threshold", ylabel="Expectation Value", legend=:topleft)

UndefVarError: UndefVarError: `expectations` not defined

**An important note about this notebook:**
The runtime and errors for different combinations of observable, circuit, and initial state vary **tremedously**. Truly challenging simulations may require `min_abs_coefficient` below `1e-6` or `max_weight` significantly above `10`. This will require heavy compute and perhaps new truncation rules or Pauli merging strategies (but this is left for another tutorial).